# Tutorial 4: Save and load Compose

## Overview

While constructing and using a flow, it's convenient to introspect the inner architecture of the flow to understand how it work and to ensure that we plug-n-play the right component into the flow. This capability can prove even more usefulness for those who don't build the flow and just use those that are packaged and distributed by other people.

Any flow constructed with `theflow.Composable` allow users to:

- View values of all current nodes and params.
- View the definition of any nodes and params.
- Check if any object is compatible with the flow and can be plugged into the flow.
- Visualize the flow architecture.

Suppose we have a flow `Plus` that consists of component `Sum1` and `Sum2` as follow:

In [1]:
import yaml
from pprint import pprint

from theflow import Composable, Param, Node, load


def callback(obj, type_):
    return obj.a * 2


class Sum1(Composable):
    a: int
    b: int = 10
    c: int = 10
    d: int = Param(depends_on="b", default_callback=lambda obj, type_: obj.b * 2)

    def run(self) -> int:
        return self.a + self.b + self.c


class Sum2(Composable):
    a: int

    def run(self, a, b: int, *args, **kwargs) -> int:
        return self.a + a + b


class Plus(Composable):
    a: int
    e: int
    x: Composable
    y: Composable = Node(default=Sum1, default_kwargs={"a": 100})
    m: Composable = Node(default=Sum2, default_kwargs={"a": 100})

    @Param.decorate()
    def f(self):
        return self.a + self.e

    def run(self) -> int:
        x, y, m = self.x(), self.y(), self.m()
        print(f"{x=}, {y=}, {z=}")
        return x + y + z

In [16]:
from typing import Any

In [14]:
import inspect

In [23]:
from theflow.utils.modules import import_dotted_string

In [5]:
from typing import Any

In [8]:
import typing

In [9]:
typing.Any.__qualname__

AttributeError: '_SpecialForm' object has no attribute '__qualname__'

In [7]:
Any.__module__, Any.__qualname__

AttributeError: '_SpecialForm' object has no attribute '__qualname__'

In [2]:
from pprint import pprint
step = Plus(a=20, e=20, x=Sum1(a=20))
xyz = step.describe()
pprint(xyz)

Error exporting theflow.base.Param.default_callback: Cannot serialize lambda functions
Error exporting theflow.base.Param.default_callback: Cannot serialize lambda functions


{'nodes': {'m': {'__type__': 'theflow.base.Node',
                 'default': {'nodes': {},
                             'params': {'a': {'__type__': 'param',
                                              'default': '{{ '
                                                         'theflow.base.empty '
                                                         '}}',
                                              'default_callback': None,
                                              'depends_on': None,
                                              'help': '',
                                              'no_cache': False,
                                              'refresh_on_set': False,
                                              'strict_type': False}},
                             'type': '__main__.Sum2'},
                 'default_callback': None,
                 'default_kwargs': {'a': 100},
                 'depends_on': None,
                 'help': '',
                 'input

In [11]:
from typing import _SpecialForm

def inspect_output(obj):
    for key, value in obj.items():
        if isinstance(value, _SpecialForm):
            import pdb; pdb.set_trace()
        print(key, type(value))
        if isinstance(value, dict):
            inspect_output(value)

In [13]:
inspect_output(xyz)

type <class 'str'>
params <class 'dict'>
f <class 'dict'>
__type__ <class 'str'>
default <class 'str'>
default_callback <class 'str'>
help <class 'str'>
refresh_on_set <class 'bool'>
strict_type <class 'bool'>
depends_on <class 'NoneType'>
no_cache <class 'bool'>
a <class 'dict'>
__type__ <class 'str'>
default <class 'str'>
default_callback <class 'NoneType'>
help <class 'str'>
refresh_on_set <class 'bool'>
strict_type <class 'bool'>
depends_on <class 'NoneType'>
no_cache <class 'bool'>
e <class 'dict'>
__type__ <class 'str'>
default <class 'str'>
default_callback <class 'NoneType'>
help <class 'str'>
refresh_on_set <class 'bool'>
strict_type <class 'bool'>
depends_on <class 'NoneType'>
no_cache <class 'bool'>
nodes <class 'dict'>
y <class 'dict'>
__type__ <class 'str'>
type <class 'str'>
default <class 'dict'>
type <class 'str'>
params <class 'dict'>
b <class 'dict'>
__type__ <class 'str'>
default <class 'int'>
default_callback <class 'NoneType'>
help <class 'str'>
refresh_on_set <cla

ipdb>  type(a)


*** NameError: name 'a' is not defined


ipdb>  type(value)


<class 'typing._SpecialForm'>


...

'a'


ipdb>  value


typing.Any


ipdb>  c


a <class 'typing._SpecialForm'>
b <class 'type'>
output <class 'str'>
x <class 'dict'>
__type__ <class 'str'>
type <class 'str'>
default <class 'str'>
default_kwargs <class 'dict'>
default_callback <class 'NoneType'>
help <class 'str'>
depends_on <class 'NoneType'>
no_cache <class 'bool'>
input <class 'str'>
output <class 'str'>


In [22]:
func = xyz['params']['f']['default_callback']
print(func)

<function Param.decorate.<locals>.inner.<locals>.<lambda> at 0x7fc5b6143a60>


In [16]:
step = Plus(a=20, e=20, x=Sum1(a=20))
print(yaml.dump(step.dump(), sort_keys=False))

type: __main__.Plus
params:
  a: 20
  e: 20
  f: 40
nodes:
  m:
    type: __main__.Sum2
    params:
      a: 100
    nodes: {}
  x:
    type: __main__.Sum1
    params:
      a: 20
      b: 10
      c: 10
    nodes: {}
  y:
    type: __main__.Sum1
    params:
      a: 100
      b: 10
      c: 10
    nodes: {}



In [3]:
step.e(step, None)

40

In [9]:
step.e

<function __main__.callback(obj, type_)>

In [13]:
modules = {
    "__main__.Plus": Plus,
    "__main__.Sum2": Sum2,
    "__main__.Sum1": Sum1,
}

step2 = load(step.dump(), safe=True, allowed_modules=modules)

Module __main__.callback is not allowed. Allowed modules are ['__main__.Plus', '__main__.Sum2', '__main__.Sum1']


In [14]:
step2.e

AttributeError: Parameter e is not set and has no default value

In [8]:
step2.a = 100

In [10]:
step2.e(step, None)

40

In [3]:
print(d)

{'type': '__main__.Plus', 'params': {'a': 1000}, 'nodes': {'m': {'type': '__main__.Sum2', 'params': {'a': 100}, 'nodes': {}}, 'x': {'type': '__main__.Sum1', 'params': {'a': 20, 'b': 10, 'c': 10}, 'nodes': {}}, 'y': {'type': '__main__.Sum1', 'params': {'a': 100, 'b': 10, 'c': 10}, 'nodes': {}}}}


In [4]:
from theflow.base import compose_from_dict
step2 = compose_from_dict(d)

In [9]:
step2.export()

{'type': '__main__.Plus',
 'params': {'a': 1000},
 'nodes': {'m': {'type': '__main__.Sum2', 'params': {'a': 100}, 'nodes': {}},
  'x': {'type': '__main__.Sum1',
   'params': {'a': 20, 'b': 10, 'c': 10},
   'nodes': {}},
  'y': {'type': '__main__.Sum1',
   'params': {'a': 100, 'b': 10, 'c': 10},
   'nodes': {}}}}

## View values of current nodes and params

`theflow` allows viewing all values of current nodes and params with `.describe()`. This method returns a dictionary contains name and value of all nodes and params.

In [3]:
from theflow.utils.paths import import_dotted_string

In [4]:
cl = import_dotted_string("__main__.Plus")

## View the definition of any node and param

`theflow` allows getting the definition of a node or a param with `.specs(...)`:

In [3]:
print("Node m")
pprint(step.specs("m"))
print("-" * 10)
print("Param x.b")
pprint(step.specs("x.b"))
print("-" * 10)

Node m
{'default': <class '__main__.Sum2'>,
 'default_callback': None,
 'default_kwargs': {'a': 100},
 'depends_on': None,
 'help': '',
 'input': {'a': typing.Any, 'b': <class 'int'>},
 'no_cache': False,
 'output': <class 'int'>,
 'type': 'node'}
----------
Param x.b
{'__type__': 'param',
 'default': 10,
 'default_callback': None,
 'depends_on': None,
 'help': '',
 'no_cache': False,
 'refresh_on_set': False,
 'strict_type': False}
----------


## Check if any object is compatible with the flow

`theflow` will inspect the method `.run`'s input and output signature to determine if a component can be plug-n-play to an existing flow.

In [4]:
from typing import List

class WrongInputName(Composable):
    def run(self, x, b: int, *args, **kwargs):
        return 10

class MissingInputVariable(Composable):
    def run(self, b: int, *args, **kwargs):
        return 10

class WrongInputType(Composable):
    def run(self, a, b: List[int], *args, **kwargs):
        return 10

class WrongOutputType(Composable):
    def run(self, a, b: int, *args, **kwargs) -> str:
        return "10"

class CorrectExact(Composable):
    def run(self, a, b: int, *args, **kwargs) -> int:
        return 10

class CorrectExtraArgs(Composable):
    def run(self, a, b: int, x, *args, **kwargs) -> int:
        return 10

class CorrectMissingTypeAssumeTypeAny(Composable):
    def run(self, a, b, x, *args, **kwargs):
        return 10

print(f"{step.is_compatible('m', WrongInputName)=}")
print(f"{step.is_compatible('m', MissingInputVariable)=}")
print(f"{step.is_compatible('m', WrongInputType)=}")
print(f"{step.is_compatible('m', WrongOutputType)=}")
print(f"{step.is_compatible('m', CorrectExact)=}")
print(f"{step.is_compatible('m', CorrectExtraArgs)=}")
print(f"{step.is_compatible('m', CorrectMissingTypeAssumeTypeAny)=}")

step.is_compatible('m', WrongInputName)=False
step.is_compatible('m', MissingInputVariable)=False
step.is_compatible('m', WrongInputType)=False
step.is_compatible('m', WrongOutputType)=False
step.is_compatible('m', CorrectExact)=True
step.is_compatible('m', CorrectExtraArgs)=True
step.is_compatible('m', CorrectMissingTypeAssumeTypeAny)=True


-------